In [2]:
pip install seaborn matplotlib plotly scikit-learn pandas numpy nltk

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached contourpy-1.3.1-cp313-cp313-win_amd64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.56.0-cp313-cp313-win_amd64.whl.metadata (103 kB)
  Using cached kiwisolver-1.4.8-cp313-cp313-win_amd64.whl.metadata (6.3 kB)
  Using cached pillow-11.1.0-cp313-cp313-win_amd64.whl.metadata (9.3 kB)
  Using cached pyparsing-3.2.1-py3-none-any.whl.metadata (5.0 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ----------------------------------


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Exception:
Traceback (most recent call last):
  File "D:\Jupyter\Google_PlayStore\venv\Lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "D:\Jupyter\Google_PlayStore\venv\Lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ~~~~~~~~~~~~~^^^^^
  File "D:\Jupyter\Google_PlayStore\venv\Lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ~~~~~~~~~~~~~^^^^^
  File "D:\Jupyter\Google_PlayStore\venv\Lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
                  ~~~~~~~~~~~~~~^^^^^
  File "E:\Program Files\Lib\http\client.py", line 479, in read
    s = self.fp.rea

In [3]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import webbrowser
import os

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
nltk.download('vader_lexicon')

In [ ]:
# 1. load the dataset

apps_df = pd.read_csv("D:/Jupyter/Google_PlayStore/Data/PlayStore Data.csv")
reviews_df = pd.read_csv("D:/Jupyter/Google_PlayStore/Data/User reviews.csv")

In [ ]:
# 2. Data Cleaning

apps_df = apps_df.dropna(subset=['Rating'])
for column in apps_df.columns:
    apps_df[column].fillna(apps_df[column].mode()[0])
apps_df.drop_duplicates(inplace=True)    
apps_df = apps_df[apps_df['Rating']<=5]
reviews_df.dropna(subset=['Translated_Review'],inplace=True)

In [ ]:
# Merge datasets on 'App' and handle non-matching apps
merged_df = pd.merge(apps_df,reviews_df,on='App',how='inner')

In [ ]:
# 3. Data Transformation

apps_df['Reviews'] = apps_df['Reviews'].astype(int)
apps_df['Installs']=apps_df['Installs'].str.replace(',','').str.replace('+','').astype(int)
apps_df['Price']=apps_df['Price'].str.replace('$','').astype(float)

In [ ]:
def convert_size(size):
    if isinstance(size, str): 
        if 'M' in size:
            return float(size.replace('M', ''))
        elif 'k' in size:
            return float(size.replace('k', '')) / 1024 
        elif 'G' in size:
            return float(size.replace('G', '')) * 1024 
        else:
            return None  
    return size 

In [ ]:
apps_df.loc[:, 'Size'] = apps_df['Size'].apply(convert_size)

In [ ]:
# Logarthmic
# Add log_installs and log_reviews columns

apps_df['Log_Installs'] = np.log(apps_df['Installs'])
apps_df['Log_Reviews'] = np.log(apps_df['Reviews'])

In [ ]:
# Add rating group column

def rating_group(rating):
    if rating >=4:
        return 'Top rated app'
    elif rating >=3:
        return 'Above average'
    else:
        return 'Below average'
apps_df['Rating_Group']=apps_df['Rating'].apply(rating_group)        

In [ ]:
# revenue column
apps_df['Revenue']=apps_df['Price']*apps_df['Installs']

In [ ]:
# Sentiment Analysis

sia = SentimentIntensityAnalyzer()
reviews_df['Sentiment_Score'] = reviews_df['Translated_Review'].apply(lambda x: sia.polarity_scores(str(x))['compound'])

In [ ]:
apps_df['Last Updated']=pd.to_datetime(apps_df['Last Updated'],errors = 'coerce')
apps_df['Year']=apps_df['Last Updated'].dt.year

In [ ]:
import numpy as np

# Ensure the required columns exist
if 'Log Reviews' not in apps_df.columns:
    if 'Reviews' in apps_df.columns:
        apps_df['Log Reviews'] = np.log1p(apps_df['Reviews'])  # log(1 + Reviews) to handle zeros
    else:
        raise KeyError("'Reviews' column is missing in the dataset.")

if 'Log Installs' not in apps_df.columns:
    if 'Installs' in apps_df.columns:
        apps_df['Log Installs'] = np.log1p(apps_df['Installs'])  # log(1 + Installs) to handle zeros
    else:
        raise KeyError("'Installs' column is missing in the dataset.")

if 'Price' not in apps_df.columns:
    raise KeyError("'Price' column is missing in the dataset.")

if 'Rating' not in apps_df.columns:
    raise KeyError("'Rating' column is missing in the dataset.")


In [ ]:
import tkinter as tk
import tkinter.ttk as ttk 
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg


class AppDashboard(tk.Tk):
    def __init__(self):
        super().__init__() 
        self.title("Google Play Store Analysis Dashboard")
        self.geometry("1200x800") 
        self.configure(bg="lightgray") 

        # Create a main frame
        main_frame = tk.Frame(self)
        main_frame.pack(fill=tk.BOTH, expand=True)  

        # Create a main canvas and scrollbars
        canvas = tk.Canvas(main_frame, bg='lightgray')
        v_scrollbar = ttk.Scrollbar(main_frame, orient="vertical", command=canvas.yview)  
        h_scrollbar = ttk.Scrollbar(main_frame, orient="horizontal", command=canvas.xview) 
        v_scrollbar.pack(side="right", fill="y")
        h_scrollbar.pack(side="bottom", fill="x")
        canvas.pack(side="left", fill="both", expand=True)

        # Create a scrollable frame
        scrollable_frame = ttk.Frame(canvas)  
        scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
        )

        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw") 
        canvas.config(yscrollcommand=v_scrollbar.set, xscrollcommand=h_scrollbar.set)  

       # Create and grid layout for the dashboard
        for i in range(6):
            scrollable_frame.columnconfigure(i, weight=1)

        # Create and place frames for each visualization
        self.create_category_analysis(scrollable_frame, 0, 0)
        self.create_type_analysis(scrollable_frame, 0, 1)
        self.create_rating_sentiment_analysis(scrollable_frame, 0, 2)
        self.create_installation_update_analysis(scrollable_frame, 0, 3)
        self.create_additional_insights(scrollable_frame, 0, 4)
        self.create_ml_model_evaluation(scrollable_frame, 0, 5)

    def create_category_analysis(self, parent, row, column):
        frame = ttk.Frame(parent, padding="5")
        frame.grid(row=row, column=column, sticky="nsew", pady=5)
        frame.columnconfigure(0, weight=1)

        # Assuming you have a DataFrame called 'apps_df'
        category_counts = apps_df['Category'].value_counts().nlargest(10)

        fig, ax = plt.subplots(figsize=(5, 3))
        sns.barplot(x=category_counts.values, y=category_counts.index, palette='Spectral', ax=ax)
        ax.set_title('Top Categories on Play Store')
        ax.set_xlabel('Number of Apps')
        ax.set_ylabel('Category')
        fig.tight_layout()
        canvas = FigureCanvasTkAgg(fig, master=frame)  
        canvas.draw()
        canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)  

    def create_type_analysis(self, parent, row, column):
        frame = ttk.Frame(parent, padding="5")
        frame.grid(row=row, column=column, sticky="nsew", pady=5)
        frame.columnconfigure(0, weight=1)
        type_counts = apps_df['Type'].value_counts()
        fig, ax = plt.subplots(figsize=(3, 2))
        ax.pie(type_counts, labels=type_counts.index, autopct='%1.1f%%', startangle=140, colors=['#ff9999', '#66b3ff'])
        ax.set_title('Distribution of Free and Paid Apps')
        fig.tight_layout()
        canvas = FigureCanvasTkAgg(fig, master=frame)
        canvas.draw()
        canvas.get_tk_widget().pack(expand=True, fill="both") 

    def create_rating_sentiment_analysis(self, parent, row, column):
        frame = ttk.Frame(parent, padding="5")
        frame.grid(row=row, column=column, sticky="nsew", pady=5)
        frame.columnconfigure(0, weight=1)
        fig, ax = plt.subplots(figsize=(5, 3))
        sns.histplot(apps_df['Rating'], bins=30, kde=True, color="purple", ax=ax) 
        ax.set_title("Rating Distribution")
        ax.set_xlabel('Rating')
        ax.set_ylabel('Frequency')
        fig.tight_layout()
        canvas = FigureCanvasTkAgg(fig, master=frame)
        canvas.draw()
        canvas.get_tk_widget().pack(expand=True, fill="both")  

    def create_installation_update_analysis(self, parent, row, column):
        frame = ttk.Frame(parent, padding="5")
        frame.grid(row=row, column=column, sticky="nsew", pady=5)
        frame.columnconfigure(0, weight=1)

        # Assuming 'apps_df' has 'Category' and 'Installs' columns
        installs_by_category = apps_df.groupby('Category')['Installs'].sum().nlargest(10)
        fig, ax = plt.subplots(figsize=(5, 3))
        sns.barplot(x=installs_by_category.values, y=installs_by_category.index, palette='coolwarm', ax=ax)
        ax.set_title('Installations by Category')
        ax.set_xlabel('Number of Installations')
        ax.set_ylabel('Category')
        fig.tight_layout()
        canvas = FigureCanvasTkAgg(fig, master=frame)
        canvas.draw()
        canvas.get_tk_widget().pack(expand=True, fill="both")   

        updates_over_year = apps_df['Year'].value_counts().sort_index() 
        fig, ax = plt.subplots(figsize=(5, 3))
        sns.lineplot(x=updates_over_year.index, y=updates_over_year.values, marker='o', ax=ax) 
        ax.set_title('Distribution of App Updates Over the Year')
        ax.set_xlabel('Year')
        ax.set_ylabel('Number of Updates') 
        fig.tight_layout()
        canvas = FigureCanvasTkAgg(fig, master=frame)
        canvas.draw()
        canvas.get_tk_widget().pack(expand=True, fill="both")

    def create_additional_insights(self, parent, row, column):
        frame = ttk.Frame(parent, padding="5")
        frame.grid(row=row, column=column, sticky="nsew", pady=5)
        frame.columnconfigure(0, weight=1)
        revenue_by_category = apps_df.groupby('Category')['Revenue'].sum().nlargest(10) 
        fig, ax = plt.subplots(figsize=(5, 3))
        sns.barplot(x=revenue_by_category.values, y=revenue_by_category.index, palette='viridis', ax=ax)  
        ax.set_title('Revenue by Category')
        ax.set_xlabel('Revenue')
        ax.set_ylabel('Category')
        fig.tight_layout()
        canvas = FigureCanvasTkAgg(fig, master=frame)
        canvas.draw()
        canvas.get_tk_widget().pack(expand=True, fill="both")  

        # Genres Counts
        genres_counts = apps_df['Genres'].str.split(';').explode().value_counts().nlargest(10)
        fig, ax = plt.subplots(figsize=(5, 3))
        sns.barplot(x=genres_counts.values, y=genres_counts.index, palette='magma', ax=ax)
        ax.set_title('Top Genres on Play Store')
        ax.set_xlabel('Number of Apps')
        ax.set_ylabel('Genres')
        fig.tight_layout()
        canvas = FigureCanvasTkAgg(fig, master=frame)
        canvas.draw()
        canvas.get_tk_widget().pack(expand=True, fill="both")
        # Effect ofLast Update on Rating
        fig, ax = plt.subplots(figsize=(5, 3))
        sns.boxplot(data=apps_df, x='Year', y='Rating', palette='cool', ax=ax) 
        ax.set_title('Effect of Last Update on Rating')
        ax.set_xlabel('Year')
        ax.set_ylabel('Rating')
        fig.tight_layout()
        canvas = FigureCanvasTkAgg(fig, master=frame)
        canvas.draw()
        canvas.get_tk_widget().pack(expand=True, fill="both")
        fig, ax = plt.subplots(figsize=(5, 3))
        sns.boxplot(data=apps_df, x='Type', y='Rating', palette='Set2', ax=ax)
        ax.set_title('Ratings for Paid vs Free Apps')
        ax.set_xlabel('Type')
        ax.set_ylabel('Rating')
        fig.tight_layout()
        canvas = FigureCanvasTkAgg(fig, master=frame)
        canvas.draw()
        canvas.get_tk_widget().pack(expand=True, fill="both")

    def create_ml_model_evaluation(self, parent, row, column):
        frame = ttk.Frame(parent, padding="5")
        frame.grid(row=row, column=column, sticky="nsew", pady=5)
        frame.columnconfigure(0, weight=1)

         # Ensure the required columns exist
        if 'Log Reviews' not in apps_df.columns:
            apps_df['Log Reviews'] = np.log1p(apps_df['Reviews'])  # log(1 + Reviews)
        if 'Log Installs' not in apps_df.columns:
            apps_df['Log Installs'] = np.log1p(apps_df['Installs'])  # log(1 + Installs)

        # Assuming 'apps_df' has 'Log Reviews', 'Log Installs', 'Price', and 'Rating' columns
        X = apps_df[['Log Reviews', 'Log Installs', 'Price']]  
        y = apps_df['Rating']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        model = RandomForestRegressor(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)  

         # ML Model Evaluation Plot
        fig, ax = plt.subplots(figsize=(5, 3))
        ax.scatter(y_test, y_pred, alpha=0.3) 
        ax.plot([0, 5], [0, 5], '--') 
        ax.set_title(f'ML Model Evaluation (MSE: {mse:.2f}, R2: {r2:.2f})')  
        ax.set_xlabel('True Rating')
        ax.set_ylabel('Predicted Rating')
        fig.tight_layout()
        canvas = FigureCanvasTkAgg(fig, master=frame)
        canvas.draw()
        canvas.get_tk_widget().pack(expand=True, fill="both") 
        

if __name__ == "__main__":
    app = AppDashboard()
    app.mainloop()
   